<a href="https://colab.research.google.com/github/argalusmp/CH2-PS_Recommendation-System/blob/mi/Recommender_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from typing import Dict, Text

In [ ]:
from IPython.core.interactiveshell import InteractiveShell

# Set shell to show all lines of output
InteractiveShell.ast_node_interactivity = 'all'

import json

events = []

with open('drive/MyDrive/Capstone Project/events_nd_2.json', 'r') as fin:
    # Append each line to the books
    events = [json.loads(l) for l in fin]

events = [event for event in events]
print(f'Found {len(events)} event.')

Found 2786 event.


In [ ]:
[event for event in events][:10]

[{'Event Name': 'Volunteers Needed For Rise Up & Stay Put! Home Rescue Fair',
  'Category': 'Youth Development',
  'Qualifications': ['youth mentoring', 'youth empowerment'],
  'Domisili': 'Solo',
  'Age': '>18'},
 {'Event Name': 'Web designer',
  'Category': 'Youth Development',
  'Qualifications': ['mentoring', 'team management', 'public relation'],
  'Domisili': 'Jawa Barat',
  'Age': '>20'},
 {'Event Name': 'Urban Adventures - Ice Skating at Lasker Rink',
  'Category': 'Youth Development',
  'Qualifications': ['art therapy', 'tutoring'],
  'Domisili': 'Jawa Barat',
  'Age': 'All'},
 {'Event Name': 'Fight global hunger and support women farmers - Join the Oxfam Action Corps in NYC!',
  'Category': 'Youth Development',
  'Qualifications': ['event planning',
   'event coordination',
   'public relation'],
  'Domisili': 'Indonesia',
  'Age': '16-38'},
 {'Event Name': "Stop 'N' Swap",
  'Category': 'Youth Development',
  'Qualifications': ['volunteer coordination'],
  'Domisili': 'Maluk

In [ ]:
event_index = {event["Event Name"]: idx for idx, event in enumerate(events)}
index_event = {idx: event for event, idx in event_index.items()}

event_category_index = {category["Category"]: idx for idx, category in enumerate(events)}
index_event_category = {idx: category for category, idx in event_category_index.items()}

# event_Q_index = {skills["Qualifications"]: idx for idx, skills in enumerate(events)}
# index_event_Q = {idx: skills for skills, idx in event_Q_index.items()}

In [ ]:
event_category_index["Pet and Animal Service"]
# index_event_category[43]

2785

In [ ]:
from itertools import chain

Qualifications = list(chain(*[event["Qualifications"] for event in events]))
print(f"There are {len(set(Qualifications))} unique skills.")

There are 90 unique skills.


In [ ]:
print(Qualifications)

['youth mentoring', 'youth empowerment', 'mentoring', 'team management', 'public relation', 'art therapy', 'tutoring', 'event planning', 'event coordination', 'public relation', 'volunteer coordination', 'communication', 'leadership training', 'content creation', 'public speaking', 'leadership development', 'public speaking', 'event planning', 'public relation', 'communication', 'communication', 'event planning', 'health education', 'public relation', 'communication', 'team building', 'team building', 'environmental education', 'event planning', 'communication', 'teaching', 'volunteer coordination', 'environmental education', 'team building', 'art therapy', 'event coordination', 'english language tutoring', 'volunteer coordination', 'public speaking', 'public speaking', 'public speaking', 'leadership development', 'event coordination', 'leadership development', 'team building', 'public relation', 'leadership development', 'team building', 'art therapy', 'leadership development', 'publi

In [ ]:
Qualifications_related = [q for q in Qualifications if q in event_index.keys()]
print(f"There are {len(set(Qualifications_related))} unique skills to other event.")

There are 0 unique skills to other event.


In [ ]:
Qualifications_related

[]


In [ ]:
from collections import Counter, OrderedDict

def count_items(l):
    """Return ordered dictionary of counts of objects in `l`"""

    # Create a counter object
    counts = Counter(l)

    # Sort by highest count first and place in ordered dictionary
    counts = sorted(counts.items(), key = lambda x: x[1], reverse = True)
    counts = OrderedDict(counts)

    return counts

In [ ]:
unique_Q = list(chain(*[list(set(event["Qualifications"])) for event in events]))
print(f"There are {len(set(unique_Q))} unique skills.")

Q_counts = count_items(unique_Q)
list(Q_counts.items())[:10]

There are 117 unique skills.


[('public relation', 180),
 ('communication', 172),
 ('graphic design', 162),
 ('administrative tasks', 142),
 ('emergency response', 140),
 ('wildlife rehabilitation', 139),
 ('computer literacy', 137),
 ('customer service', 127),
 ('visual comunication', 127),
 ('creative design', 126)]

In [ ]:
Qualifications = [skills.lower() for skills in unique_Q]
print(f"There are {len(set(Qualifications))} unique skills.")

Q_counts = count_items(Qualifications)
list(Q_counts.items())[:10]

There are 115 unique skills.


[('public relation', 180),
 ('communication', 172),
 ('graphic design', 162),
 ('administrative tasks', 142),
 ('emergency response', 140),
 ('wildlife rehabilitation', 139),
 ('computer literacy', 137),
 ('customer service', 127),
 ('visual comunication', 127),
 ('creative design', 126)]

In [ ]:
to_remove = ['public relation', 'communication', 'graphic design', 'administrative tasks', 'emergency response', 'wildlife rehabilitation']
for t in to_remove:
    Qualifications.remove(t)
    _ = Q_counts.pop(t)

skills = Q_counts.items()
print(len(skills))

111


In [ ]:
# Find set of book wikilinks for each book
unique_qualifications_events = list(chain(*[list(set(skills for skills in event["Qualifications"] if skills in event_index.keys())) for event in events]))

# Count the number of books linked to by other books
qualifications_event_counts = count_items(unique_qualifications_events)
list(qualifications_event_counts.items())[:10]

[]

In [ ]:
skills_index = {skills: idx for idx, skills in enumerate(skills)}
index_skills = {idx: skills for skills, idx in skills_index.items()}

skills_index['pet grooming']
# index_skills[65]
# print(f'There are {len(skills_index)} wikilinks that will be used.')

In [ ]:
pairs = []

# Iterate through each event
for event in events:
    # Iterate through the links in the book
    pairs.extend((event_index[event["Event_id"]], skills_index[skills]) for skills in event["Qualifications"] if skills.lower() in skills)

len(pairs), len(skills), len(events)
pairs[76]

In [ ]:
index_event[pairs[5000][0]], index_skills[pairs[5000][1]]

# Preproccesing using Tokenizer ==========================================

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

user_ds = pd.read_csv("drive/MyDrive/Capstone Project/user_ds.csv")
event_ds= pd.read_csv("drive/MyDrive/Capstone Project/event_ds.csv")

df_user = pd.DataFrame(user_ds)
df_event = pd.DataFrame(event_ds)

print(f"found {len(df_event)} event")

df_user = df_user.astype(str)
df_event = df_event.astype(str)

# Preprocess event data
event_text = df_event['Qualifications 1'] + ' ' + df_event['Qualifications 2'] + ' ' + df_event['Qualifications 3'] + ' ' + df_event['Category']
event_text = event_text.str.lower()

# Tokenize
tokenizer = Tokenizer()
tokenizer.fit_on_texts(event_text)

event_sequences = tokenizer.texts_to_sequences(event_text)
event_data = pad_sequences(event_sequences)

# Print information about each word
word_index = tokenizer.word_index
for word, index in word_index.items():
    print(f"Word: {word}, Index: {index}")

# Optionally, you can print the total number of words in the vocabulary
total_words = len(word_index)
print(f"Total number of words in the vocabulary: {total_words}")


found 5599 event
Word: animal, Index: 1
Word: pet, Index: 2
Word: development, Index: 3
Word: event, Index: 4
Word: social, Index: 5
Word: service, Index: 6
Word: environmental, Index: 7
Word: communication, Index: 8
Word: healthcare, Index: 9
Word: youth, Index: 10
Word: management, Index: 11
Word: organizer, Index: 12
Word: it, Index: 13
Word: enviromental, Index: 14
Word: and, Index: 15
Word: leadership, Index: 16
Word: education, Index: 17
Word: conservation, Index: 18
Word: design, Index: 19
Word: public, Index: 20
Word: support, Index: 21
Word: data, Index: 22
Word: lab, Index: 23
Word: medical, Index: 24
Word: adoption, Index: 25
Word: training, Index: 26
Word: camping, Index: 27
Word: planning, Index: 28
Word: team, Index: 29
Word: rehabilitation, Index: 30
Word: assistance, Index: 31
Word: research, Index: 32
Word: community, Index: 33
Word: outreach, Index: 34
Word: relation, Index: 35
Word: health, Index: 36
Word: computer, Index: 37
Word: visual, Index: 38
Word: behavior, I

In [ ]:
index_to_count = 3
count_of_index = sum(sequence.count(index_to_count) for sequence in event_data)

AttributeError: ignored

In [ ]:
unique_skills = list(chain(*[list(set(tokenizer)) for event in events]))
print(f"There are {len(set(unique_Q))} unique skills.")

Q_counts = count_items(unique_Q)
list(Q_counts.items())[:10]

In [ ]:
print(f'Found {len(events)} event.')

Found 2786 event.


In [ ]:
word_index['pet_and_animal'] = word_index['pet']
word_index.pop('pet', None)
word_index.pop('and', None)
word_index.pop('animal', None)

NameError: ignored

In [ ]:
event_text = events['Qualifications'] + ' ' + events['Category'] + ' ' + events['Domisili']
event_text = event_text.str.lower()

TypeError: ignored